<a href="https://colab.research.google.com/github/trentfridey/qosf-screening/blob/master/Task2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install pennylane
!pip install pennylane-cirq

     |████████████████████████████████| 1.4MB 3.5MB/s 


In [19]:
import pennylane as qml
import numpy as np
from pennylane_cirq import ops

In [20]:
# Definitions

observable = qml.Hermitian(np.array([[1,0,0,0],[0,0,2,0],[0,2,0,0],[0,0,0,1]]), wires=[0,1])

# Device setup

dev = qml.device('cirq.mixedsimulator', wires=2, shots=1000, analytic=False)

@qml.qnode(dev)
def ansatz(thetas):
  qml.RY(thetas[0], wires=0)
  qml.RX(thetas[1], wires=1)
  qml.CNOT(wires=[0,1])
  # Here we add in the BitFlip operator for modelling noise
  ops.BitFlip(0.5, wires=0)
  ops.BitFlip(0.5, wires=1)
  return qml.expval(observable)

def cost(thetas):
  return 2 - ansatz(thetas)

# Optimization

steps = 100

init_params = np.random.normal(0, np.pi, 2)

params = init_params

opt = qml.GradientDescentOptimizer(stepsize=0.4)

for i in range(steps):
  params = opt.step(cost, params)
  if i % 10 == 0:
    print("Step {} Cost {}".format(i, cost(params)))

print("Optimized angles: {} {}".format(params[0]/np.pi, params[1]/np.pi))




Step 0 Cost 2.182
Step 10 Cost 0.488
Step 20 Cost 0.5
Step 30 Cost 0.478
Step 40 Cost 0.5
Step 50 Cost 0.5149999999999999
Step 60 Cost 0.4950000000000001
Step 70 Cost 0.474
Step 80 Cost 0.5069999999999999
Step 90 Cost 0.502
Optimized angles: 0.5048645834573443 -1.539118300786664
